In [1]:
import torch
import torch.nn as nn
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from data import DataGenerator

data = DataGenerator()


In [3]:
train_data = data.train

In [4]:
train_data

,uid,mid,rating
0,0,1192,1
1,0,660,1
2,0,913,1
3,0,3407,1
4,0,2354,1
...,...,...,...
1000204,6039,1090,1
1000205,6039,1093,1
1000206,6039,561,1
1000207,6039,1095,1


In [5]:
class NCF(nn.Module):
    def __init__(self, num_users, num_likes, embed_size, num_hidden, output_size):
        super(NCF, self).__init__()
        self.user_emb = nn.Embedding(num_users, embed_size)
        self.like_emb = nn.Embedding(num_likes,embed_size)
        self.fc1 = nn.Linear(embed_size*2, num_hidden[0])
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(num_hidden[0], num_hidden[1])
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(num_hidden[1], num_hidden[2])
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(num_hidden[2], num_hidden[3])
        self.relu4 = nn.ReLU()
        self.outLayer = nn.Linear(num_hidden[3], output_size)
        self.out_act = nn.Sigmoid()

    def forward(self, u, v):
        U = self.user_emb(u)
        V = self.like_emb(v)
        out = torch.cat([U,V], dim=1)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        out = self.relu4(out)
        out = self.outLayer(out)
        out = self.out_act(out)
        return out


In [6]:
emb_size = 128
hidden_layers = np.array([emb_size, 64, 32, 16])
output_size = 1
num_epochs = 1
learning_rate = 0.001
batch_size = 2048
num_negatives = 5

random_samples = 100
top_K = 10

preTrained_NCF = NCF(data.num_users, data.num_movies, emb_size, hidden_layers, output_size).to(device)

In [7]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(preTrained_NCF.parameters(), lr=learning_rate, weight_decay=1e-6)
preTrained_NCF.train()

NCF(
  (user_emb): Embedding(6040, 128)
  (like_emb): Embedding(3952, 128)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (relu4): ReLU()
  (outLayer): Linear(in_features=16, out_features=1, bias=True)
  (out_act): Sigmoid()
)

In [8]:
num_batches = np.int64(np.floor(train_data.shape[0] / batch_size))
'''TRAIN MODEL'''
for i in range(num_epochs):
    j = 0
    for batch in np.array_split(train_data, num_batches):
        batch_df = data.add_negatives(batch[['uid', 'mid', 'rating']], n_samples=4)
        users = torch.LongTensor(batch_df.uid.to_numpy()).to(device)
        items = torch.LongTensor(batch_df.mid.to_numpy()).to(device)
        ratings = torch.FloatTensor(batch_df.rating.to_numpy()).to(device)
        y_hat = preTrained_NCF(users, items)
        loss = criterion(y_hat, ratings.unsqueeze(1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'epoch: {i + 1} \nbatch: {j} out of: {num_batches} \naverage loss: {loss.item()}\n')
        j += 1

epoch: 1 
batch: 0 out of: 485 
average loss: 0.6260589361190796

epoch: 1 
batch: 1 out of: 485 
average loss: 0.6183147430419922

epoch: 1 
batch: 2 out of: 485 
average loss: 0.6109557151794434

epoch: 1 
batch: 3 out of: 485 
average loss: 0.6005444526672363

epoch: 1 
batch: 4 out of: 485 
average loss: 0.5967397689819336

epoch: 1 
batch: 5 out of: 485 
average loss: 0.5918882489204407

epoch: 1 
batch: 6 out of: 485 
average loss: 0.5858030915260315

epoch: 1 
batch: 7 out of: 485 
average loss: 0.5710885524749756

epoch: 1 
batch: 8 out of: 485 
average loss: 0.5681321024894714

epoch: 1 
batch: 9 out of: 485 
average loss: 0.553505539894104

epoch: 1 
batch: 10 out of: 485 
average loss: 0.5428939461708069

epoch: 1 
batch: 11 out of: 485 
average loss: 0.531600832939148

epoch: 1 
batch: 12 out of: 485 
average loss: 0.5277777314186096

epoch: 1 
batch: 13 out of: 485 
average loss: 0.5100416541099548

epoch: 1 
batch: 14 out of: 485 
average loss: 0.4909614324569702

epoch: 

In [9]:
# %% model evaluation: hit rate and NDCG
test_data = data.test

In [ ]:
import heapq
import math

def evaluate_model(model, df_val:DataGenerator, top_K, random_samples):
    model.eval()
    avg_HR = np.zeros((len(df_val), top_K))
    avg_NDCG = np.zeros((len(df_val), top_K))

    for i in range(len(df_val)):
        test_df = data.add_negatives(df_val, n_samples=random_samples)
        users, items = torch.LongTensor(test_df.uid).to(device), torch.LongTensor(test_df.mid).to(device)
        y_hat = model(users, items)

        y_hat = y_hat.cpu().detach().numpy().reshape((-1,))
        test_item_input = items.cpu().detach().numpy().reshape((-1,))
        map_item_score = {}
        for j in range(len(y_hat)):
            map_item_score[test_item_input[j]] = y_hat[j]
        for k in range(top_K):
            # Evaluate top rank list
            ranklist = heapq.nlargest(k, map_item_score, key=map_item_score.get)
            gtItem = test_item_input[0]
            for item in ranklist:
                if item==gtItem:
                    avg_HR[i, k] = 1
                    avg_NDCG[i, k] = math.log(2) / math.log(i+2)
                else:
                    avg_HR[i, k] = 0
                    avg_NDCG[i, k] = 0
    avg_HR = np.mean(avg_HR, axis=0)
    avg_NDCG = np.mean(avg_NDCG, axis=0)
    return avg_HR, avg_NDCG


In [ ]:
torch.save(preTrained_NCF.state_dict(), "models/preTrained_NCF")

In [ ]:
avg_HR_preTrain, avg_NDCG_preTrain = evaluate_model(preTrained_NCF, test_data, top_K, random_samples)


In [10]:
# debiased_NCF = NCF(data.num_users, data.num_movies, emb_size, hidden_layers, output_size).to(device)
# debiased_NCF.load_state_dict(torch.load("trained-models/preTrained_NCF"))
# debiased_NCF.to(device)

In [11]:
import constants
import pandas as pd


def features():
    df = pd.read_csv('MovieLens/users.dat',
                     sep='::',
                     header=None,
                     names=['uid', 'gender', 'age', 'job', 'zip'],
                     engine='python')
    df.drop(columns=['uid'], inplace=True)
    df.index.rename('uid', inplace=True)
    df.gender = pd.get_dummies(df.gender, drop_first=True)  # 0:F, 1:M
    return df.reset_index()


features = features()

# complete = pd.merge(df, data.df, how=' outer', on='uid')
# complete.drop(columns=['date', 'latest', 'zip'], inplace=True)
# gender_embed = compute_gender_direction(train_data, train_protected_attributes, users_embed)
# S = 0 indicates male and S = 1 indicates female

In [12]:
# see constants for more info
drop = [0, 10, 13, 19]

clean = features[~features['job'].isin(drop)]

clean['rating'] = 1
num_users = clean.uid.nunique()
# num_movies = clean.mid.nunique()
num_jobs = clean.job.nunique()

new_job_index = np.arange(num_jobs)
item_id = clean[['job']].drop_duplicates()
item_id['njob'] = np.arange(num_jobs)
clean = pd.merge(clean, item_id, on=['job'], how='left')
clean.job = clean.njob
clean.head()

/var/folders/kf/j1jq_1qs78n4gd1vczc8vfw80000gn/T/ipykernel_6498/2444525869.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean['rating'] = 1


,uid,gender,age,job,zip,rating,njob
0,1,1,56,0,70072,1,0
1,2,1,25,1,55117,1,1
2,3,1,45,2,02460,1,2
3,4,1,25,3,55455,1,3
4,5,0,50,4,55117,1,4


In [13]:
msk = np.random.rand(len(clean)) < 0.7

train = clean[msk]
test = clean[~msk]

clean.info()  # 1476+3444 = 4920


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4920 entries, 0 to 4919
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   uid     4920 non-null   int64 
 1   gender  4920 non-null   uint8 
 2   age     4920 non-null   int64 
 3   job     4920 non-null   int64 
 4   zip     4920 non-null   object
 5   rating  4920 non-null   int64 
 6   njob    4920 non-null   int64 
dtypes: int64(5), object(1), uint8(1)
memory usage: 273.9+ KB


In [14]:
''' GET USER EMBEDDING '''
user_embeds = preTrained_NCF.user_emb.weight.data.cpu().detach().numpy()
user_embeds = user_embeds.astype('float')

In [15]:
''' COMPUTE GENDER EMBEDDING '''
gender_embed = np.zeros((2,user_embeds.shape[1]))
num_users_x_group = np.zeros((2, 1))

for i in range(train.shape[0]):
    u = train['uid'].iloc[i]
    if train['gender'].iloc[i] == 0:
        gender_embed[0] +=  user_embeds[u]
        num_users_x_group[0] += 1.0
    else:
        gender_embed[1] +=  user_embeds[u]
        gender_embed[1] += 1.0
        num_users_x_group[1] += 1.0

In [16]:
''' VERTICAL BIAS'''
gender_embed = gender_embed / num_users_x_group
# vBias = compute_bias_direction(gender_embed)
vBias = gender_embed[1].reshape((1,-1)) - gender_embed[0].reshape((1,-1))
vBias = vBias / np.linalg.norm(vBias,axis=1,keepdims=1)

vBias

array([[0.09069008, 0.09003054, 0.08496672, 0.09050297, 0.08642624,
        0.08902275, 0.08749835, 0.08371646, 0.08727884, 0.08833783,
        0.08636001, 0.08821293, 0.08301546, 0.08872426, 0.08411755,
        0.08561655, 0.09076883, 0.08932683, 0.08813156, 0.08828874,
        0.08777421, 0.09261886, 0.09256089, 0.08925772, 0.08927288,
        0.08415085, 0.09027174, 0.09163169, 0.08620748, 0.09127901,
        0.08907101, 0.08760005, 0.0856063 , 0.08866908, 0.09112992,
        0.09213509, 0.08922896, 0.09195355, 0.08909875, 0.086764  ,
        0.09079708, 0.08806218, 0.08690775, 0.08625827, 0.08981551,
        0.0859188 , 0.08947847, 0.08731626, 0.08268248, 0.09064306,
        0.08455186, 0.08849131, 0.09175542, 0.08973892, 0.08846167,
        0.08708216, 0.08771133, 0.08641088, 0.08827489, 0.08764957,
        0.08548956, 0.09011761, 0.09402284, 0.08968159, 0.08752014,
        0.08969738, 0.09025029, 0.08867402, 0.08841823, 0.08767961,
        0.08769625, 0.08736225, 0.08915547, 0.08

In [17]:
''' LINEAR PROJECTION '''
debiased_user_embeds = user_embeds


for i in range(len(clean)):
    u = clean['uid'].iloc[i]
    debiased_user_embeds[u] = user_embeds[u] - (np.inner(user_embeds[u].reshape(1,-1),vBias)[0][0])*vBias


In [18]:
'''UPDATE USER EMBEDDINGS'''
fairness_thres = torch.tensor(0.1).to(device)
epsilonBase = torch.tensor(0.0).to(device)

n_careers = clean.job.nunique()
# replace page items with career items
preTrained_NCF.like_emb = nn.Embedding(n_careers,emb_size).to(device)
# freeze user embedding
preTrained_NCF.user_emb.weight.requires_grad=False

# replace user embedding of the model with debiased embeddings
preTrained_NCF.user_emb.weight.data = torch.from_numpy(debiased_user_embeds.astype(np.float32)).to(device)


In [19]:
'''optimize'''
# fair_fine_tune_model(DF_NCF,train_data, num_epochs, learning_rate,batch_size,num_negatives,n_careers,train_gender,fairness_thres,epsilonBase, unsqueeze=True)

criterion = nn.BCELoss()
optimizer = optim.Adam(preTrained_NCF.parameters(), lr=learning_rate, weight_decay=1e-6)

preTrained_NCF.train()

torch.save(preTrained_NCF.state_dict(), "models/DF_NCF")

In [20]:
all_users = torch.LongTensor(train['uid'].values).to(device)
all_items = torch.LongTensor(train['job'].values).to(device)
all_genders = torch.LongTensor(train['gender'].values).to(device)

from fairness_measures import Measures

m = Measures()
num_batches = np.int64(np.floor(train.shape[0] / batch_size))

for i in range(num_epochs):
    j = 1
    for batch in np.array_split(train, num_batches):
        batch_df = data.add_negatives(
            df=batch[['uid', 'job', 'rating']],
            item='job',
            items=set(clean.job.unique()),
            n_samples=4
        )
        print(batch_df.head())
        users = torch.LongTensor(batch_df.uid.to_numpy()).to(device)
        items = torch.LongTensor(batch_df.job.to_numpy()).to(device)
        ratings = torch.FloatTensor(batch_df.rating.to_numpy()).to(device)
        print(items)
        y_hat = preTrained_NCF(users, items)

        loss1 = criterion(y_hat, ratings.unsqueeze(1))

        predicted_probs = preTrained_NCF(all_users, all_items)
        avg_epsilon = m.computeEDF(all_genders,predicted_probs,n_careers,all_items,device)
        print(avg_epsilon)
        #criteroin hinge
        loss2 = torch.max(torch.tensor(0.0).to(device), (avg_epsilon-epsilonBase))

        loss = loss1 + fairness_thres*loss2

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'epoch: {i + 1} \nbatch: {j} out of: {num_batches} \naverage loss: {loss.item()}\n')
        j+=1

   uid  job  rating
0    0    7       0
1    0   10       0
2    0   12       0
3    0   14       0
4    1    0       1
tensor([ 7, 10, 12,  ...,  1,  5,  5])
tensor(0.0662, grad_fn=<MeanBackward0>)
epoch: 1 
batch: 1 out of: 1 
average loss: 47.4340705871582



In [24]:
torch.save(preTrained_NCF.state_dict(), "models/DF_NCF")

In [28]:
import heapq
def evaluate_fine_tune(model,df_val,top_K,random_samples):
    model.eval()
    avg_HR = np.zeros((len(df_val),top_K))
    avg_NDCG = np.zeros((len(df_val),top_K))

    # for i in range(len(df_val)):
    test_df = data.add_negatives(
        df_val,
        item='job',
        items=set(clean.job.unique()),
        n_samples=random_samples
    )
    users, items = torch.LongTensor(test_df.uid).to(device), torch.LongTensor(test_df.job).to(device)
    y_hat = model(users, items)

    y_hat = y_hat.cpu().detach().numpy().reshape((-1,))
    items = items.cpu().detach().numpy().reshape((-1,))
    map_item_score = {}
    for j in range(len(y_hat)):
        map_item_score[items[j]] = y_hat[j]
    for k in range(top_K):
        # Evaluate top rank list
        ranklist = heapq.nlargest(k, map_item_score, key=map_item_score.get)
        gtItem = items[0]
        avg_HR[i,k] = getHitRatio(ranklist, gtItem)
        avg_NDCG[i,k] = getNDCG(ranklist, gtItem)
    avg_HR = np.mean(avg_HR, axis = 0)
    avg_NDCG = np.mean(avg_NDCG, axis = 0)
    return avg_HR, avg_NDCG

def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

In [29]:
emb_size = 128
hidden_layers = np.array([emb_size, 64, 32, 16])
output_size = 1
num_epochs = 10
learning_rate = 0.001
batch_size = 256
num_negatives = 5

random_samples = 15
top_K = 10

avg_HR_DF_NCF, avg_NDCG_DF_NCF = evaluate_fine_tune(preTrained_NCF, test, top_K, random_samples)

In [30]:
avg_HR_DF_NCF

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [31]:
def fairness_measures(model,df_val,num_items):
    model.eval()
    users, items = torch.LongTensor(df_val.uid.to_numpy()).to(device), torch.LongTensor(df_val.job.to_numpy()).to(device)
    y_hat = model(users, items)

    avg_epsilon = m.computeEDF(all_genders,y_hat,num_items,items,device)
    U_abs = m.compute_absolute_unfairness(all_genders,y_hat,num_items,items,device)

    avg_epsilon = avg_epsilon.cpu().detach().numpy().reshape((-1,)).item()
    print(f"average differential fairness: {avg_epsilon: .3f}")

    U_abs = U_abs.cpu().detach().numpy().reshape((-1,)).item()
    print(f"absolute unfairness: {U_abs: .3f}")


In [32]:
fairness_measures(preTrained_NCF,test,n_careers)


average differential fairness:  0.188
absolute unfairness:  0.081
